# Feature Engineering Workshop
## DSE 200 - Day 3

### Learning Objectives
By the end of this workshop, you will understand:
1. **Temporal Feature Engineering**: How to capture cyclic patterns in time-series data
2. **Geospatial Feature Engineering**: How to represent location data for prediction tasks
3. **Categorical Feature Engineering**: When numbers have meaning beyond their value
4. **Text Feature Engineering**: How to convert text into numerical features using TF-IDF

### Why Feature Engineering?
Linear and polynomial regression have limitations. Feature engineering helps us:
- Capture patterns that simple models can't
- Reduce overfitting by creating meaningful features
- Improve model performance dramatically

In [ ]:
%pip install pandas numpy scikit-learn matplotlib seaborn geopy requests

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from geopy.distance import geodesic
import requests
import os

np.random.seed(42)
plt.rcParams['figure.figsize'] = [12, 6]
sns.set_style('whitegrid')

---
## Part 1: Temporal Feature Engineering
### The Challenge: Oscillating Temperature Data

**Scenario:** You have hourly temperature data for August. Temperature follows a daily cycle - it rises during the day and falls at night. How can we capture this pattern?

**Key Insight:** Linear regression draws a straight line. Polynomial regression can overfit. But temperature is cyclic - what goes up must come down!

### Learning Points:
- Sin/Cos encoding captures cyclic patterns
- Lag features use past values to predict future
- Rolling statistics smooth out noise

In [ ]:
data_url = 'https://dse200.dev/Day3/hourly_temperatures_august.csv'
temp_file = 'hourly_temperatures_august.csv'

if not os.path.exists(temp_file):
    r = requests.get(data_url)
    with open(temp_file, 'wb') as f:
        f.write(r.content)

df_temp = pd.read_csv(temp_file)
df_temp['timestamp'] = pd.to_datetime(df_temp['timestamp'])

print("Temperature Dataset:")
print(df_temp.head())
print(f"\nShape: {df_temp.shape}")

### Task 1.1: Visualize the Raw Data
Plot temperature vs time to observe the pattern.

In [ ]:
# TODO: Create a line plot of temperature over time
# Hint: Use plt.plot(df_temp['timestamp'], df_temp['temperature_f'])



### Task 1.2: Baseline - Linear Regression
Try predicting temperature using only a time index. This should perform poorly!

In [ ]:
df_temp['time_index'] = range(len(df_temp))

split_idx = int(len(df_temp) * 0.7)
train_df = df_temp[:split_idx]
test_df = df_temp[split_idx:]

# TODO: Train a linear regression model
# X_train = train_df[['time_index']].values
# y_train = train_df['temperature_f'].values
# X_test = test_df[['time_index']].values
# y_test = test_df['temperature_f'].values

# model = LinearRegression()
# model.fit(X_train, y_train)
# y_pred = model.predict(X_test)

# TODO: Calculate and print MSE and R²
# mse = mean_squared_error(y_test, y_pred)
# r2 = r2_score(y_test, y_pred)

# TODO: Plot actual vs predicted


### Task 1.3: Polynomial Regression
Try using polynomial features. Experiment with different degrees (2, 3, 5, 7).

In [ ]:
# TODO: Create polynomial features and train models with different degrees
# Hint: Use PolynomialFeatures from sklearn.preprocessing
# degrees = [2, 3, 5, 7]
# For each degree:
#   - Create polynomial features
#   - Train model
#   - Calculate MSE and R²
#   - Plot results



### Task 1.4: Feature Engineering - Cyclic Encoding
Now let's engineer better features! Temperature follows a 24-hour cycle. We'll use sin/cos to capture this.

In [ ]:
df_temp_fe = df_temp.copy()

# TODO: Create cyclic features for hour of day
# hour_sin = np.sin(2 * np.pi * df_temp_fe['timestamp'].dt.hour / 24)
# hour_cos = np.cos(2 * np.pi * df_temp_fe['timestamp'].dt.hour / 24)

# TODO: Add these as new columns to df_temp_fe

# TODO: Train linear regression with these cyclic features
# Use both hour_sin and hour_cos as features

# TODO: Calculate and print MSE and R²

# TODO: Plot actual vs predicted


---
## Part 2: Geospatial Feature Engineering
### The Challenge: House Prices and Location

**Scenario:** You're predicting house prices in San Diego. Houses close to downtown or UCSD are more expensive. But how do we represent location?

**Key Insight:** Latitude/longitude have some linear relationship with price when there's ONE center. But with multiple centers (downtown + UCSD), we need distance features!

### Learning Points:
- Raw lat/lon might not work well
- Distance to important locations is meaningful
- Multiple centers require multiple distance features

In [ ]:
data_url = 'https://dse200.dev/Day3/real_estate_san_diego.csv'
re_file = 'real_estate_san_diego.csv'

if not os.path.exists(re_file):
    r = requests.get(data_url)
    with open(re_file, 'wb') as f:
        f.write(r.content)

df_re = pd.read_csv(re_file)

print("Real Estate Dataset:")
print(df_re.head())
print(f"\nShape: {df_re.shape}")

### Task 2.1: Visualize Geographic Distribution
Create a scatter plot of house locations, colored by price.

In [ ]:
# TODO: Create scatter plot with longitude on x-axis, latitude on y-axis
# Color points by price
# Add markers for San Diego downtown (32.715736, -117.161087) and UCSD (32.8812, -117.2344)



### Task 2.2: Baseline - Linear Regression with Lat/Lon
Try predicting price using just latitude and longitude.

In [ ]:
split_idx = int(len(df_re) * 0.7)
train_df = df_re[:split_idx]
test_df = df_re[split_idx:]

# TODO: Train linear regression with latitude and longitude
# X_train = train_df[['latitude', 'longitude']].values
# y_train = train_df['price'].values

# TODO: Calculate and print MSE, RMSE, and R²

# TODO: Create scatter plot of actual vs predicted prices


### Task 2.3: Feature Engineering - Distance Features
Calculate distance from each house to important centers (San Diego downtown and UCSD).

In [ ]:
san_diego_coords = (32.715736, -117.161087)
ucsd_coords = (32.8812, -117.2344)

# TODO: Calculate distances using geopy.distance.geodesic
# For each row in df_re:
#   - Calculate distance to San Diego downtown
#   - Calculate distance to UCSD
#   - Calculate minimum distance to either center

# distances_sd = []
# distances_ucsd = []
# for idx, row in df_re.iterrows():
#     dist_sd = geodesic(san_diego_coords, (row['latitude'], row['longitude'])).miles
#     dist_ucsd = geodesic(ucsd_coords, (row['latitude'], row['longitude'])).miles
#     distances_sd.append(dist_sd)
#     distances_ucsd.append(dist_ucsd)

# TODO: Add distance columns to df_re
# df_re['distance_san_diego'] = distances_sd
# df_re['distance_ucsd'] = distances_ucsd
# df_re['min_distance'] = np.minimum(distances_sd, distances_ucsd)


### Task 2.4: Visualize Distance vs Price
Plot how price relates to distance from centers.

In [ ]:
# TODO: Create scatter plots:
# 1. Distance from San Diego vs Price
# 2. Distance from UCSD vs Price
# 3. Minimum distance vs Price



### Task 2.5: Train Model with Distance Features
Now train a linear regression using the minimum distance feature.

In [ ]:
# TODO: Split data with distance feature
# TODO: Train linear regression with min_distance
# TODO: Calculate and print MSE, RMSE, and R²
# TODO: Compare with baseline (lat/lon only)
# TODO: Plot actual vs predicted



---
## Part 3: Categorical Feature Engineering
### The Challenge: I-5 Traffic Volume

**Scenario:** Predict vehicle volume on I-5 by hour. Is hour 5 really "1 less" than hour 6? Or does "rush hour" vs "non-rush hour" matter more?

**Key Insight:** Sometimes numbers have greater meaning than their value. Hour 7, 8, 9, 16, 17, 18 are all "rush hour" - they're categorically similar!

### Learning Points:
- Manual binning based on domain knowledge
- Clustering to discover hidden categories
- One-hot encoding for categorical features

In [ ]:
data_url = 'https://dse200.dev/Day3/i5_traffic_data.csv'
traffic_file = 'i5_traffic_data.csv'

if not os.path.exists(traffic_file):
    r = requests.get(data_url)
    with open(traffic_file, 'wb') as f:
        f.write(r.content)

df_traffic = pd.read_csv(traffic_file)

print("Traffic Dataset:")
print(df_traffic.head())
print(f"\nShape: {df_traffic.shape}")

### Task 3.1: Visualize Traffic by Hour
Create a scatter plot of vehicle volume by hour.

In [ ]:
# TODO: Create scatter plot of hour vs vehicle_volume
# What patterns do you see?



### Task 3.2: Baseline - Linear Regression
Try predicting volume using just the hour number.

In [ ]:
split_idx = int(len(df_traffic) * 0.7)
train_df = df_traffic[:split_idx]
test_df = df_traffic[split_idx:]

# TODO: Train linear regression with hour
# TODO: Calculate and print MSE, RMSE, MAE, and R²
# TODO: Plot actual vs predicted



### Task 3.3: Manual Binning - Rush Hour Feature
Create a binary feature: is_rush_hour (1 if hour is 7-9 or 16-18, 0 otherwise).

In [ ]:
# TODO: Create rush hour feature
# df_traffic['is_rush_hour'] = ((df_traffic['hour'] >= 7) & (df_traffic['hour'] <= 9)) | \
#                               ((df_traffic['hour'] >= 16) & (df_traffic['hour'] <= 18))
# df_traffic['is_rush_hour'] = df_traffic['is_rush_hour'].astype(int)

# TODO: Visualize average volume by rush hour status



### Task 3.4: Train Model with Rush Hour Feature
Train linear regression using both hour and is_rush_hour.

In [ ]:
# TODO: Split data
# TODO: Train linear regression with ['hour', 'is_rush_hour']
# TODO: Calculate and print metrics
# TODO: Compare with baseline



### Task 3.5: Clustering - Discover Hidden Categories
Use K-Means clustering to automatically discover traffic patterns.

**Steps:**
1. Use Elbow Method to find optimal number of clusters
2. Cluster based on hour, volume, and speed
3. Create one-hot encoded features for clusters
4. Train model with cluster features

In [ ]:
# TODO: Prepare data for clustering
# clustering_features = ['hour', 'vehicle_volume', 'avg_speed_mph']
# X_cluster = df_traffic[clustering_features].values

# TODO: Standardize features
# scaler = StandardScaler()
# X_scaled = scaler.fit_transform(X_cluster)

# TODO: Elbow method - try k from 2 to 10
# inertias = []
# K_range = range(2, 11)
# for k in K_range:
#     kmeans = KMeans(n_clusters=k, random_state=42)
#     kmeans.fit(X_scaled)
#     inertias.append(kmeans.inertia_)

# TODO: Plot elbow curve



In [ ]:
# TODO: Choose optimal k (around 4) and fit KMeans
# optimal_k = 4
# kmeans = KMeans(n_clusters=optimal_k, random_state=42)
# df_traffic['traffic_cluster'] = kmeans.fit_predict(X_scaled)

# TODO: Create one-hot encoded features
# cluster_dummies = pd.get_dummies(df_traffic['traffic_cluster'], prefix='cluster')
# df_traffic_clustered = pd.concat([df_traffic, cluster_dummies], axis=1)

# TODO: Visualize clusters
# Create scatter plots colored by cluster



In [ ]:
# TODO: Train model with cluster features
# features = ['hour'] + [col for col in df_traffic_clustered.columns if col.startswith('cluster_')]
# TODO: Calculate and print metrics
# TODO: Compare with baseline and manual binning



---
## Part 4: Text Feature Engineering with TF-IDF
### The Challenge: Airbnb Listing Prices

**Scenario:** Predict Airbnb listing price based on the listing name/description. Text can't be fed directly into models - we need to convert it to numbers!

**Key Insight:** TF-IDF (Term Frequency-Inverse Document Frequency) measures how important a word is to a document. Words like "luxury" or "beachfront" should increase price!

### Learning Points:
- TF-IDF converts text to numerical features
- Rare, descriptive words are more valuable
- Text features can be powerful predictors

In [ ]:
data_url = 'https://dse200.dev/Day3/airbnb_sd_listings.csv'
airbnb_file = 'airbnb_sd_listings.csv'

if not os.path.exists(airbnb_file):
    r = requests.get(data_url)
    with open(airbnb_file, 'wb') as f:
        f.write(r.content)

df_airbnb = pd.read_csv(airbnb_file)
df_airbnb = df_airbnb.dropna(subset=['price', 'name'])
df_airbnb = df_airbnb[df_airbnb['price'] > 0]

print("Airbnb Dataset:")
print(df_airbnb.head())
print(f"\nShape: {df_airbnb.shape}")
print("\nSample listing names:")
for i, name in enumerate(df_airbnb['name'].head(10), 1):
    print(f"{i}. {name}")

### Task 4.1: Explore Text Data
Look at the distribution of listing names and prices.

In [ ]:
# TODO: Plot price distribution
# TODO: Show statistics of name length
# df_airbnb['name_length'] = df_airbnb['name'].str.len()



### Task 4.2: Create TF-IDF Features
Convert listing names to TF-IDF features.

In [ ]:
# TODO: Initialize TfidfVectorizer
# tfidf = TfidfVectorizer(
#     max_features=100,      # Keep top 100 words
#     stop_words='english',  # Remove common words like 'the', 'a', 'is'
#     ngram_range=(1, 2)     # Use single words and pairs
# )

# TODO: Fit and transform the text
# tfidf_features = tfidf.fit_transform(df_airbnb['name'])

# TODO: Convert to DataFrame
# tfidf_df = pd.DataFrame(
#     tfidf_features.toarray(),
#     columns=[f'tfidf_{word}' for word in tfidf.get_feature_names_out()]
# )

# TODO: Combine with original data
# df_airbnb_tfidf = pd.concat([df_airbnb.reset_index(drop=True), tfidf_df], axis=1)

print(f"Number of TF-IDF features created: {tfidf_df.shape[1]}")
print("\nTop 20 TF-IDF features:")
print(list(tfidf.get_feature_names_out()[:20]))

### Task 4.3: Analyze Feature Importance
Find which words correlate most with price.

In [ ]:
# TODO: Calculate correlation between each TF-IDF feature and price
# feature_importance = []
# for col in tfidf_df.columns:
#     corr = abs(df_airbnb_tfidf[col].corr(df_airbnb_tfidf['price']))
#     feature_importance.append((col, corr))

# TODO: Sort by correlation
# feature_importance.sort(key=lambda x: x[1], reverse=True)

# TODO: Print top 15 features
# TODO: Visualize as bar chart



### Task 4.4: Train Model with TF-IDF Features
Use TF-IDF features to predict price.

In [ ]:
# TODO: Split data (use random shuffle for text data!)
# df_shuffled = df_airbnb_tfidf.sample(frac=1, random_state=42).reset_index(drop=True)
# split_idx = int(len(df_shuffled) * 0.7)

# TODO: Get TF-IDF columns
# tfidf_cols = [col for col in df_airbnb_tfidf.columns if col.startswith('tfidf_')]

# TODO: Train linear regression
# TODO: Calculate and print metrics
# TODO: Plot actual vs predicted
# TODO: Show top words by coefficient magnitude

